# test embeddings

plot tsne of the embeddings extracted by the CNN model trained on the dataset.

-> write a module for input generation. given the folder and file names get NxTxM matrix
-> each data point, load model in eval mode. 
-> forward the given 

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision.transforms as transforms
import sys
from ret_benchmark.config import cfg
from ret_benchmark.modeling import build_model
import os
import numpy as np
import re
from sklearn.manifold import TSNE
import pickle
import matplotlib.pyplot as plt
from itertools import combinations

os.environ['CUDA_VISIBLE_DEVICES'] = '3' # gpu to run the code
sys.path.insert(1, '/data/home/shruti/voxceleb/motion_signature/research-ms-loss-master/')
config_path = 'configs/ms_resnet50_fabnet_vox_test.yaml'
cache_emb_file = '/data/home/shruti/motion_signature/obj/ms_resnet50_fabnet_vox_test' # this file is cache for fabnet metric learning embeddings on leader data


In [2]:
# get all the embedding
def load_obj(fldr, name ):
    if not os.path.exists(os.path.join(fldr, name + '.pkl')):
        return None
    with open(os.path.join(fldr, name + '.pkl'), 'rb') as f:
        return pickle.load(f)

# save the obj
def save_obj(obj, fldr, name ):
    os.makedirs(fldr, exist_ok=True)
    with open(os.path.join(fldr, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

## load model

In [3]:
def load_model(cfgpath):
    
    cfg.merge_from_file(cfgpath)
    
    # load the model
    model = build_model(cfg)
    device = torch.device(cfg.MODEL.DEVICE)
    model.to(device)
    model.eval()
    return model

In [4]:
model = load_model(config_path)

Resuming from model ...


## load data and labels

In [5]:
subfldrs = np.array(['bo_faceswap', 'bo_UWfake', 'diff_test', 'dt_faceswap', 'hc_faceswap', 'bs_faceswap',
                        'ew_faceswap', 'dt_imposter', 'hc_imposter', 'bs_imposter', 'ew_imposter', 'bo_imposter', 'trump_fake'])

def get_fabnet_ldrs_data():
        
    # read mean and std files
    means = np.reshape(np.load(cfg.INPUT.MEAN), (1, -1))
    stds = np.reshape(np.load(cfg.INPUT.STD), (1, -1))

    # the test files of leaders
    root = os.path.dirname(cfg.DATA.TEST_IMG_SOURCE)
    path_list = []
    label_list = []
    print(cfg.DATA.TRAIN_IMG_SOURCE)
    with open(cfg.DATA.TRAIN_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
            
    print(cfg.DATA.TEST_IMG_SOURCE)
    with open(cfg.DATA.TEST_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    test_file = cfg.DATA.TEST_IMG_SOURCE.replace('_val', '_test')
    print(test_file)
    with open(test_file, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    # add the files from 
    ofset = np.max(label_list) + 1
    for j in range(len(subfldrs)):
        _path = [os.path.join(subfldrs[j], f) for f in os.listdir(os.path.join(root, subfldrs[j])) if f.endswith('.npy')]
        _label = list(np.zeros((len(_path), ), dtype='int')+ofset+j)
        path_list.extend(_path)
        label_list.extend(_label)
    
    data = {}
    data['mean'] = means
    data['std'] = stds
    data['root'] = root
    data['path_list'] = path_list
    data['label_list'] = label_list
    
    return data

def get_vgg_ldrs_data():
        
    # read mean and std files
    means = np.reshape(np.load(cfg.INPUT.MEAN), (1, -1))
    stds = np.reshape(np.load(cfg.INPUT.STD), (1, -1))

    # the test files of leaders
    root = os.path.dirname(cfg.DATA.TEST_IMG_SOURCE)
    path_list = []
    label_list = []
    print(cfg.DATA.TRAIN_IMG_SOURCE)
    with open(cfg.DATA.TRAIN_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
            
    print(cfg.DATA.TEST_IMG_SOURCE)
    with open(cfg.DATA.TEST_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    test_file = cfg.DATA.TEST_IMG_SOURCE.replace('_val', '_test')
    print(test_file)
    with open(test_file, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    # add the files from 
    ofset = np.max(label_list) + 1
    for j in range(len(subfldrs)):
        _path = [f for f in os.listdir(root) if f.endswith('.npy') and f[:len(subfldrs[j])+1] == subfldrs[j]+'_']
        _label = list(np.zeros((len(_path), ), dtype='int')+ofset+j)
        path_list.extend(_path)
        label_list.extend(_label)
    
    data = {}
    data['mean'] = means
    data['std'] = stds
    data['root'] = root
    data['path_list'] = path_list
    data['label_list'] = label_list
    
    return data

In [6]:
data_files = get_fabnet_ldrs_data()
#data_files = get_vgg_ldrs_data()
print(len(data_files['path_list']))

/data/home/shruti/voxceleb/fabnet/leaders/leaders_150_train.txt
/data/home/shruti/voxceleb/fabnet/leaders/leaders_150_val.txt
/data/home/shruti/voxceleb/fabnet/leaders/leaders_150_test.txt
4329


# get metric embedding

for every video file:
    1) load the npy
    2) for every 150 window with 5 frame step, pass it through the model and get the embedding
    3) in the save the embedding and the label in a dictionary
    4) save the embeddings 

In [7]:

feat_pair = list(combinations(range(20), 2))
feat_pair = np.array([[j[0], j[1]] for j in feat_pair])
def aus190_lbl(in_feat):
    
    aus = in_feat[:, :20]
    corr = np.corrcoef(aus[:, feat_pair[:, 0]].T, 
                       aus[:, feat_pair[:, 1]].T)[0:len(feat_pair), 
                                                  len(feat_pair):].diagonal()
        
    if np.sum(np.isnan(corr))>0:
        return None
    
    return np.reshape(corr, (1, -1)).copy()


def FabNet(in_feat):
    
    fab_emb = in_feat/np.linalg.norm(in_feat, axis=1, keepdims=True)
    return (fab_emb-data_files['mean'])/data_files['std']

# dataset loader
data_loader_dict = {'aus190_lbl': aus190_lbl, 'FabNet': FabNet}

In [8]:
def feature_extract(infile):
    
    st = 5; emb_sz = cfg.MODEL.HEAD.DIM
    try:
        fab_emb = np.load(infile)
    except Exception as e:
        print(e)
        return None
    
    # if not enough time window
    if len(fab_emb)<cfg.INPUT.FRAME_LENGTH:
        return None
    
    if len(fab_emb.shape)>2:
        fab_emb = fab_emb[:, :, 0].copy()
    
    feat_rng = np.arange(0, len(fab_emb)-cfg.INPUT.FRAME_LENGTH+1, st)
    out_feat = np.zeros((len(feat_rng), emb_sz)) + np.nan
    
    print(f' {out_feat.shape}')
    for i in range(len(feat_rng)):
        cur_data = data_loader_dict[cfg.INPUT.DATA_LOADER](fab_emb[feat_rng[i]:feat_rng[i]+cfg.INPUT.FRAME_LENGTH, :])
        
        if cur_data is None:
            continue
        
        #transform
        cur_data = np.reshape(cur_data, (1, 1, cur_data.shape[0], cur_data.shape[1]))
        tensor_data = torch.from_numpy(cur_data).cuda().float()

        # pass the model
        with torch.no_grad():
            out_feat[i, :] = model(tensor_data).data.cpu().numpy().ravel()
    
    # remove nans
    out_feat = out_feat[np.sum(np.isnan(out_feat), axis=1)<1, :].copy()
    return out_feat

In [9]:
int2str_lbl = {0:'bo', 1:'br', 2:'bs', 3:'cb', 4:'dt', 5:'ew', 6:'hc', 7:'jb', 8:'kh', 9:'pb', 
               10:'bo_faceswap', 11:'bo_UWfake', 12:'diff_test', 13:'dt_faceswap', 14:'hc_faceswap', 15:'bs_faceswap',
               16:'ew_faceswap', 17:'dt_imposter', 18:'hc_imposter', 19:'bs_imposter', 20:'ew_imposter', 
               21:'bo_imposter', 22:'trump_fake'}

for lbl in int2str_lbl.keys():
    
    # for every embedding file
    cur_dict = load_obj( cache_emb_file, f'{cfg.INPUT.FRAME_LENGTH}_{int2str_lbl[lbl]}')    
    if cur_dict is None:

        cur_dict = {}
        cur_files = np.array(data_files['path_list'])[np.array(data_files['label_list'])==lbl]
        for i in range(len(cur_files)): #

            print(cur_files[i], end=' ')
            # 
            cur_feat = feature_extract(os.path.join(data_files['root'], cur_files[i]))
            if cur_feat is not None:
                cur_dict[cur_files[i]] = cur_feat

        save_obj(cur_dict, cache_emb_file, f'{cfg.INPUT.FRAME_LENGTH}_{int2str_lbl[lbl]}')

bo/--QxAtXZJpQ_0.npy  (262, 512)
bo/--QxAtXZJpQ_1.npy  (296, 512)
bo/--QxAtXZJpQ_2.npy  (295, 512)
bo/--QxAtXZJpQ_3.npy  (295, 512)
bo/-9mSTKUWS7U_0.npy  (242, 512)
bo/-9mSTKUWS7U_1.npy  (300, 512)
bo/-9mSTKUWS7U_2.npy  (298, 512)
bo/-9mSTKUWS7U_3.npy  (300, 512)
bo/-LhfOv1-wOQ_0.npy  (242, 512)
bo/-LhfOv1-wOQ_1.npy  (300, 512)
bo/-LhfOv1-wOQ_2.npy  (300, 512)
bo/-LhfOv1-wOQ_3.npy  (107, 512)
bo/-ZJvRzOdSCw_0.npy  (267, 512)
bo/-ZJvRzOdSCw_1.npy  (293, 512)
bo/-ZJvRzOdSCw_2.npy  (295, 512)
bo/-ZJvRzOdSCw_3.npy  (109, 512)
bo/-my9hVoGv2M_0.npy  (265, 512)
bo/-my9hVoGv2M_1.npy  (299, 512)
bo/-my9hVoGv2M_2.npy  (300, 512)
bo/-my9hVoGv2M_3.npy  (300, 512)
bo/-my9hVoGv2M_4.npy  (295, 512)
bo/-my9hVoGv2M_5.npy  (114, 512)
bo/0SaVqB0w718_0.npy  (241, 512)
bo/0SaVqB0w718_1.npy  (300, 512)
bo/0SaVqB0w718_2.npy  (299, 512)
bo/0SaVqB0w718_3.npy  (254, 512)
bo/0T_Jv4Kbgek_0.npy  (237, 512)
bo/0T_Jv4Kbgek_1.npy  (300, 512)
bo/0T_Jv4Kbgek_2.npy  (292, 512)
bo/0_9ue3Tx8Jk_0.npy  (267, 512)
bo/0_9ue3T

bo/DAi7--S3Y5k_1.npy  (286, 512)
bo/DAi7--S3Y5k_2.npy  (200, 512)
bo/Dt46A8dvRSs_0.npy  (239, 512)
bo/Dt46A8dvRSs_1.npy  (299, 512)
bo/Dt46A8dvRSs_2.npy  (298, 512)
bo/Dt46A8dvRSs_3.npy  (99, 512)
bo/Dxsdp3r5Dek_0.npy  (265, 512)
bo/Dxsdp3r5Dek_1.npy  (295, 512)
bo/Dxsdp3r5Dek_2.npy  (300, 512)
bo/Dxsdp3r5Dek_3.npy  (115, 512)
bo/E3gfMumXCjI_0.npy  (236, 512)
bo/E3gfMumXCjI_1.npy  (295, 512)
bo/E3gfMumXCjI_2.npy  (299, 512)
bo/E3gfMumXCjI_3.npy  (254, 512)
bo/EAZIHIiuhrc_0.npy  (242, 512)
bo/EAZIHIiuhrc_1.npy  (300, 512)
bo/EAZIHIiuhrc_2.npy  (301, 512)
bo/EAZIHIiuhrc_3.npy  (85, 512)
bo/EMyqZ8lGgXY_0.npy  (236, 512)
bo/EMyqZ8lGgXY_1.npy  (300, 512)
bo/EMyqZ8lGgXY_2.npy  (294, 512)
bo/EMyqZ8lGgXY_3.npy  (139, 512)
bo/Eh4f3it89BU_0.npy  (269, 512)
bo/Eh4f3it89BU_1.npy  (300, 512)
bo/Eh4f3it89BU_2.npy  (296, 512)
bo/Eh4f3it89BU_3.npy  (298, 512)
bo/Eh4f3it89BU_4.npy  (21, 512)
bo/FDa3Gmq36wM_0.npy  (268, 512)
bo/FDa3Gmq36wM_1.npy  (300, 512)
bo/FDa3Gmq36wM_2.npy  (300, 512)
bo/FDa3Gmq36w

bo/Tpaj9Sm7i7I_1.npy  (299, 512)
bo/Tpaj9Sm7i7I_2.npy  (300, 512)
bo/Tpaj9Sm7i7I_3.npy  (299, 512)
bo/Tpaj9Sm7i7I_4.npy  (36, 512)
bo/V1fA9-SryzY_0.npy  (266, 512)
bo/V1fA9-SryzY_1.npy  (300, 512)
bo/V1fA9-SryzY_2.npy  (298, 512)
bo/V1fA9-SryzY_3.npy  (274, 512)
bo/V5_vUE9FlXk_0.npy  (260, 512)
bo/V5_vUE9FlXk_1.npy  (295, 512)
bo/V5_vUE9FlXk_2.npy  (233, 512)
bo/VWc_aHqaPeE_0.npy  (236, 512)
bo/VWc_aHqaPeE_1.npy  (298, 512)
bo/VWc_aHqaPeE_2.npy  (300, 512)
bo/VWc_aHqaPeE_3.npy  (128, 512)
bo/Vd7aR5PMpuQ_0.npy  (266, 512)
bo/Vd7aR5PMpuQ_1.npy  (299, 512)
bo/Vd7aR5PMpuQ_2.npy  (299, 512)
bo/Vd7aR5PMpuQ_3.npy  (301, 512)
bo/Vd7aR5PMpuQ_4.npy  (17, 512)
bo/WQM6VG5F3Ek_0.npy  (236, 512)
bo/WQM6VG5F3Ek_1.npy  (299, 512)
bo/WQM6VG5F3Ek_2.npy  (298, 512)
bo/WsypI6avDEc_0.npy  (241, 512)
bo/WsypI6avDEc_1.npy  (299, 512)
bo/WsypI6avDEc_2.npy  (300, 512)
bo/WsypI6avDEc_3.npy  (297, 512)
bo/WsypI6avDEc_4.npy  (272, 512)
bo/XhPjeL3GQto_0.npy  (266, 512)
bo/XhPjeL3GQto_1.npy  (300, 512)
bo/XhPjeL3GQ

bo/li1aHjjqh3w_2.npy  (295, 512)
bo/li1aHjjqh3w_3.npy  (300, 512)
bo/li1aHjjqh3w_4.npy  (300, 512)
bo/li1aHjjqh3w_5.npy  (297, 512)
bo/li1aHjjqh3w_6.npy  (283, 512)
bo/lp02L6Opojw_0.npy  (241, 512)
bo/lp02L6Opojw_1.npy  (301, 512)
bo/lp02L6Opojw_2.npy  (298, 512)
bo/mJZY3oqQnMI_0.npy  (268, 512)
bo/mJZY3oqQnMI_1.npy  (299, 512)
bo/mJZY3oqQnMI_2.npy  (299, 512)
bo/mJZY3oqQnMI_3.npy  (243, 512)
bo/mnilkdlOrBI_0.npy  (239, 512)
bo/mnilkdlOrBI_1.npy  (299, 512)
bo/mnilkdlOrBI_2.npy  (300, 512)
bo/mnilkdlOrBI_3.npy  (71, 512)
bo/mox4InKEwgU_0.npy  (266, 512)
bo/mox4InKEwgU_1.npy  (299, 512)
bo/mox4InKEwgU_2.npy  (299, 512)
bo/mox4InKEwgU_3.npy  (44, 512)
bo/nGiUxYBdsFA_0.npy  (266, 512)
bo/nGiUxYBdsFA_1.npy  (300, 512)
bo/nGiUxYBdsFA_2.npy  (295, 512)
bo/nHREBzHqFTQ_0.npy  (236, 512)
bo/nHREBzHqFTQ_1.npy  (299, 512)
bo/nHREBzHqFTQ_2.npy  (300, 512)
bo/nHREBzHqFTQ_3.npy  (200, 512)
bo/ndkUaQ5HsBk_0.npy  (266, 512)
bo/ndkUaQ5HsBk_1.npy  (297, 512)
bo/ndkUaQ5HsBk_2.npy  (300, 512)
bo/ndkUaQ5Hs

br/GsvcAtrPNK8_5_1.npy  (211, 512)
br/GsvcAtrPNK8_6_0.npy  (252, 512)
br/GsvcAtrPNK8_6_1.npy  (252, 512)
br/GsvcAtrPNK8_8_0.npy  (313, 512)
br/GsvcAtrPNK8_9_0.npy  (154, 512)
br/GsvcAtrPNK8_9_1.npy  (154, 512)
br/J_3uJ3eEJHE_10_0.npy  (109, 512)
br/J_3uJ3eEJHE_11_0.npy  (201, 512)
br/J_3uJ3eEJHE_1_0.npy  (228, 512)
br/J_3uJ3eEJHE_2_0.npy  (31, 512)
br/J_3uJ3eEJHE_3_0.npy  (205, 512)
br/J_3uJ3eEJHE_3_1.npy  (205, 512)
br/J_3uJ3eEJHE_6_0.npy  (241, 512)
br/J_3uJ3eEJHE_6_1.npy  (241, 512)
br/J_3uJ3eEJHE_8_0.npy  (177, 512)
br/J_3uJ3eEJHE_8_1.npy  (177, 512)
br/J_3uJ3eEJHE_9_0.npy  (304, 512)
br/R1IPejLofGE_0_0.npy  (277, 512)
br/R1IPejLofGE_1_0.npy  (235, 512)
br/R1IPejLofGE_1_1.npy  (234, 512)
br/R1IPejLofGE_2_0.npy  (53, 512)
br/R1IPejLofGE_3_0.npy  (158, 512)
br/R1IPejLofGE_3_1.npy  (158, 512)
br/R1IPejLofGE_4_0.npy  (39, 512)
br/R1IPejLofGE_5_0.npy  (59, 512)
br/R1IPejLofGE_7_0.npy  (217, 512)
br/UUXo5S2ap-c_10_0.npy  (145, 512)
br/UUXo5S2ap-c_1_0.npy  (187, 512)
br/UUXo5S2ap-c_3_0.np

...

jb/Riln63WqR1M_0_1.npy  (179, 512)
jb/Riln63WqR1M_1_0.npy  (222, 512)
jb/Riln63WqR1M_2_0.npy  (158, 512)
jb/Riln63WqR1M_2_1.npy  (158, 512)
jb/Riln63WqR1M_3_0.npy  (265, 512)
jb/Riln63WqR1M_3_1.npy  (265, 512)
jb/Riln63WqR1M_4_0.npy  (151, 512)
jb/Riln63WqR1M_4_1.npy  (151, 512)
jb/Riln63WqR1M_5_0.npy  (307, 512)
jb/V544QfumwUY_0_0.npy  (129, 512)
jb/V544QfumwUY_1_0.npy  (256, 512)
jb/V544QfumwUY_3_0.npy  (257, 512)
jb/V544QfumwUY_5_0.npy  (260, 512)
jb/V544QfumwUY_6_0.npy  (165, 512)
jb/V544QfumwUY_7_0.npy  (122, 512)
jb/Vlu5i77oy0U_0_0.npy  (282, 512)
jb/Vlu5i77oy0U_0_1.npy  (282, 512)
jb/Vlu5i77oy0U_0_2.npy  (282, 512)
jb/Vlu5i77oy0U_1_0.npy  (216, 512)
jb/Vlu5i77oy0U_1_1.npy  (216, 512)
jb/dxQPhFOKjFQ_0_0.npy  (63, 512)
jb/dxQPhFOKjFQ_1_0.npy  (69, 512)
jb/dxQPhFOKjFQ_2_0.npy  (61, 512)
jb/dxQPhFOKjFQ_3_0.npy  (106, 512)
jb/dxQPhFOKjFQ_4_0.npy  (32, 512)
jb/dxQPhFOKjFQ_5_0.npy  (80, 512)
jb/dxQPhFOKjFQ_6_0.npy  (276, 512)
jb/dzwRVzwSwLw_10_0.npy  (99, 512)
jb/dzwRVzwSwLw_12_0.npy  

kh/eb4n37yW3sc_0_0.npy  (59, 512)
kh/eb4n37yW3sc_1_0.npy  (181, 512)
kh/eb4n37yW3sc_1_1.npy  (181, 512)
kh/eb4n37yW3sc_2_0.npy  (292, 512)
kh/eb4n37yW3sc_2_0_0.npy  (223, 512)
kh/eb4n37yW3sc_2_0_1.npy  (223, 512)
kh/eb4n37yW3sc_2_1.npy  (292, 512)
kh/eb4n37yW3sc_2_1_0.npy  (271, 512)
kh/eb4n37yW3sc_2_2_0.npy  (307, 512)
kh/eb4n37yW3sc_2_2_1.npy  (307, 512)
kh/jY0PoOGT_Pg_10_0.npy  (64, 512)
kh/jY0PoOGT_Pg_11_0.npy  (85, 512)
kh/jY0PoOGT_Pg_13_0.npy  (263, 512)
kh/jY0PoOGT_Pg_14_0.npy  (72, 512)
kh/jY0PoOGT_Pg_15_0.npy  (31, 512)
kh/jY0PoOGT_Pg_16_0.npy  (78, 512)
kh/jY0PoOGT_Pg_17_0.npy  (132, 512)
kh/jY0PoOGT_Pg_18_0.npy  (319, 512)
kh/jY0PoOGT_Pg_1_0.npy  (234, 512)
kh/jY0PoOGT_Pg_1_1.npy  (234, 512)
kh/jY0PoOGT_Pg_23_0.npy  (49, 512)
kh/jY0PoOGT_Pg_24_0.npy  (133, 512)
kh/jY0PoOGT_Pg_25_0.npy  (211, 512)
kh/jY0PoOGT_Pg_25_1.npy  (211, 512)
kh/jY0PoOGT_Pg_2_0.npy  (115, 512)
kh/jY0PoOGT_Pg_3_0.npy  (153, 512)
kh/jY0PoOGT_Pg_4_0.npy  (102, 512)
kh/jY0PoOGT_Pg_5_0.npy  (79, 512)
kh/jY0

pb/ZitWbxZwcqQ_10_0.npy  (175, 512)
pb/ZitWbxZwcqQ_1_0.npy  (224, 512)
pb/ZitWbxZwcqQ_2_0.npy  (152, 512)
pb/ZitWbxZwcqQ_3_0.npy  (55, 512)
pb/ZitWbxZwcqQ_4_0.npy  (108, 512)
pb/ZitWbxZwcqQ_5_0.npy  (91, 512)
pb/ZitWbxZwcqQ_7_0.npy  (175, 512)
pb/ZitWbxZwcqQ_7_1.npy  (175, 512)
pb/ZitWbxZwcqQ_8_0.npy  (163, 512)
pb/ZitWbxZwcqQ_9_0.npy  (151, 512)
pb/_r6R5TafbLg_12_0.npy  (268, 512)
pb/_r6R5TafbLg_16_0.npy  (211, 512)
pb/_r6R5TafbLg_16_1.npy  (211, 512)
pb/_r6R5TafbLg_1_0.npy  (77, 512)
pb/_r6R5TafbLg_20_0.npy  (250, 512)
pb/_r6R5TafbLg_26_0.npy  (222, 512)
pb/_r6R5TafbLg_26_1.npy  (222, 512)
pb/_r6R5TafbLg_2_0.npy  (125, 512)
pb/_r6R5TafbLg_4_0.npy  (180, 512)
pb/_r6R5TafbLg_5_0.npy  (69, 512)
pb/_r6R5TafbLg_7_0.npy  (90, 512)
pb/_r6R5TafbLg_8_0.npy  (92, 512)
pb/_r6R5TafbLg_9_0.npy  (183, 512)
pb/a-AZqS--VnY_12_0.npy  (42, 512)
pb/a-AZqS--VnY_13_0.npy  (138, 512)
pb/a-AZqS--VnY_1_0.npy  (41, 512)
pb/a-AZqS--VnY_2_0.npy  (65, 512)
pb/a-AZqS--VnY_3_0.npy  (69, 512)
pb/a-AZqS--VnY_9_0.np

diff_test/pQ72v_9C0U4.npy  (45, 512)
diff_test/lnCKJgarohQ.npy  (41, 512)
diff_test/mEK1olYy8XY.npy  (68, 512)
diff_test/1vpfHf42UuI.npy  (138, 512)
diff_test/oHH9KGGQsBM.npy  (191, 512)
diff_test/kzMIiQN7cVQ.npy  (66, 512)
diff_test/o-_W2gKkysw.npy  (47, 512)
diff_test/ZJ5F_C4Phds.npy  (175, 512)
diff_test/pSc8pCb5ug0.npy  (83, 512)
diff_test/mqgFOPU4PVk.npy  (50, 512)
diff_test/sZ8hFchh-i8.npy  (112, 512)
diff_test/Un5QIB0wQ-k.npy  (66, 512)
diff_test/y4PqkuhOzNc.npy  (40, 512)
diff_test/tR7z_H1qPMc.npy  (87, 512)
diff_test/tTzFkD7YJ8g.npy  (61, 512)
diff_test/IMnplOSVc8M.npy  (181, 512)
diff_test/QY4cgYNUOdI.npy  (193, 512)
diff_test/hwNn7VlnczQ.npy  (36, 512)
diff_test/g7kkCQ93QfU.npy  (54, 512)
diff_test/8fn1oZY5vYM.npy  (104, 512)
diff_test/imwcT784F8k.npy  (33, 512)
diff_test/vRyZGkbKyI4.npy  (63, 512)
diff_test/dk7BdYp5h3k.npy  (78, 512)
diff_test/oAHHbpD_uN8.npy  (52, 512)
diff_test/wCIDKwgcpfw.npy  (42, 512)
diff_test/xpr2-JQHu2M.npy  (32, 512)
diff_test/jTmquAAj8Dw.npy  (41,

dt_imposter/8vQlhWBvAwY_0_1.npy  (255, 512)
dt_imposter/4_Gf0mGJfP8_0_0.npy  (250, 512)
hc_imposter/ZwQkBfBs958_4.npy  (43, 512)
hc_imposter/VK2Mvd-nH9o_0_0.npy  (238, 512)
hc_imposter/-nQGBZQrtT0_6.npy  (109, 512)
hc_imposter/ZwQkBfBs958_0.npy  (151, 512)
hc_imposter/FlV2lOJuUPU_2.npy  (49, 512)
hc_imposter/VK2Mvd-nH9o_3.npy  (133, 512)
hc_imposter/-kjyltrKZSY_0.npy  (43, 512)
hc_imposter/6dCClwMqJK8_0.npy  (91, 512)
hc_imposter/FlV2lOJuUPU_0.npy  (55, 512)
hc_imposter/-kjyltrKZSY_8.npy  (79, 512)
hc_imposter/-kjyltrKZSY_17.npy  (79, 512)
hc_imposter/VK2Mvd-nH9o_1.npy  (55, 512)
hc_imposter/-nQGBZQrtT0_8.npy  (31, 512)
hc_imposter/-kjyltrKZSY_13.npy  (43, 512)
hc_imposter/ZwQkBfBs958_2.npy  (31, 512)
hc_imposter/-nQGBZQrtT0_12.npy  (127, 512)
hc_imposter/VK2Mvd-nH9o_0_1.npy  (238, 512)
hc_imposter/ZwQkBfBs958_5.npy  (73, 512)
hc_imposter/-kjyltrKZSY_16.npy  (55, 512)
hc_imposter/ZwQkBfBs958_1.npy  (247, 512)
hc_imposter/-kjyltrKZSY_12.npy  (31, 512)
hc_imposter/VK2Mvd-nH9o_4.npy  (241